## Library Code

In [ ]:
import boto3

cloudfront = boto3.client('cloudfront')
cloudformation = boto3.client('cloudformation')

cformation_east = boto3.client('cloudformation', region_name='us-east-1')
cformation_west = boto3.client('cloudformation', region_name='us-west-2')

gw_east = boto3.client('apigateway', region_name='us-east-1')
gw_west = boto3.client('apigateway', region_name='us-west-2')

In [ ]:
def get_stack_name(service, stage):
    return '{}-{}'.format(service,stage)

In [ ]:
def get_endpoint(cf_client, stack_name):
    response = cf_client.describe_stacks(
        StackName=stack_name
    )
    
    outputs = response['Stacks'][0]['Outputs']
    endpoint =  [d for d in outputs if d['OutputKey'] == 'ServiceEndpoint'][0]['OutputValue']
    return endpoint

In [ ]:
def form_s3_url_prefix(region):
    prefix = ''
    if region == 'us-east-1':
        prefix = 'https://s3.amazonaws.com'
    else:
        prefix = 'https://s3-' + region + '.amazonaws.com'
    return prefix

## Create the Lambda Edge Function

In [ ]:
edge_fn_stack = 'edge-rt'
cf_bucket = 'xtds-cf-templates-us-east-1'

In [ ]:
bucket_base = form_s3_url_prefix('us-east-1') + '/' + cf_bucket
print bucket_base

In [ ]:
response = cloudformation.create_stack(
    StackName=edge_fn_stack,
    TemplateURL= bucket_base + '/edge.yml',
    Parameters=[
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue':cf_bucket
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM'
    ]
)

print response

In [ ]:
print 'waiting on create of {}'.format(edge_fn_stack)
waiter = cloudformation.get_waiter('stack_create_complete')
waiter.wait(
    StackName=edge_fn_stack
)

print 'stack created'

In [ ]:
#Get the arn of the lambda function created above. We will need this to set the edge function in our
# cloud front distribution
response = cloudformation.describe_stacks(
    StackName=edge_fn_stack
)

print response

outputs = response['Stacks'][0]['Outputs']
lambda_arn = [d for d in outputs if d['OutputKey'] == 'LambdaArn'][0]['OutputValue']
print lambda_arn

## Create the API Gateway Health Checks

In [ ]:
# API service name and stage - assume uniform between regions
service = 'serverless-rest-api-with-dynamodb'
stage = 'hc1'

In [ ]:
# Health check stack names
east_hc_stack = 'east-hc'
west_hc_stack = 'west-hc'
hc_uri = '/' + stage + '/todos/health'

In [ ]:
# Get the endpoints
stack_name = get_stack_name(service, stage)
east_endpoint = get_endpoint(cformation_east, stack_name)
print east_endpoint

west_endpoint = get_endpoint(cformation_west, stack_name)
print west_endpoint

In [ ]:
from urlparse import urlparse
parsed = urlparse(east_endpoint)
east_netloc = parsed.netloc

In [ ]:
# Create the east health check
response = cloudformation.create_stack(
    StackName=east_hc_stack,
    TemplateURL= bucket_base + '/hc.yml',
    Parameters=[
        {
            'ParameterKey': 'APIEndpoint',
            'ParameterValue':east_netloc
        },
        {
            'ParameterKey': 'ResourcePath',
            'ParameterValue':hc_uri
        },
        {
            'ParameterKey':'Name',
            'ParameterValue': 'east'
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM'
    ]
)

print response

In [ ]:
print 'waiting on create of {}'.format(edge_fn_stack)
waiter = cloudformation.get_waiter('stack_create_complete')
waiter.wait(
    StackName=east_hc_stack
)

print 'stack created'

In [ ]:
response = cloudformation.describe_stacks(
    StackName=east_hc_stack
)


outputs = response['Stacks'][0]['Outputs']
east_health_check_id = [d for d in outputs if d['OutputKey'] == 'HealthCheckId'][0]['OutputValue']
print east_health_check_id

In [ ]:
parsed = urlparse(west_endpoint)
west_netloc = parsed.netloc

In [ ]:
# Create the west health check
response = cloudformation.create_stack(
    StackName=west_hc_stack,
    TemplateURL= bucket_base + '/hc.yml',
    Parameters=[
        {
            'ParameterKey': 'APIEndpoint',
            'ParameterValue':west_netloc
        },
        {
            'ParameterKey': 'ResourcePath',
            'ParameterValue':hc_uri
        },
        {
            'ParameterKey':'Name',
            'ParameterValue': 'west'
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM'
    ]
)

print response

In [ ]:
print 'waiting on create of {}'.format(edge_fn_stack)
waiter = cloudformation.get_waiter('stack_create_complete')
waiter.wait(
    StackName=west_hc_stack
)

print 'stack created'

In [ ]:
response = cloudformation.describe_stacks(
    StackName=west_hc_stack
)


outputs = response['Stacks'][0]['Outputs']
west_health_check_id = [d for d in outputs if d['OutputKey'] == 'HealthCheckId'][0]['OutputValue']
print west_health_check_id

## Lambda Function Modifications

At this point you need to go manually edit the edge function code in the AWS console to set the 
health check IDs and API endpoints in the code. They cannot be injected as environment variables at this time
due to limitations with edge functions.

Below are the parameters to use:

In [ ]:
print "const primaryHealthCheckId = '{}';".format(east_health_check_id)
print "const primaryEndpoint = '{}';".format(east_netloc)
print "const secondaryHealthCheckId = '{}';".format(west_health_check_id)
print "const secondaryEndpoint = '{}';".format(west_netloc)

### Important Note

You need to publish a new version of the lambda function after you modify it.

In [ ]:
## Make sure to update this!!!!
lambda_fn_version = '2'
cf_lambda_arn = lambda_arn + ':' + lambda_fn_version
print cf_lambda_arn

## Get the Cloud Distribution ID

In [ ]:
cloudfront_stack_name = 'cf-api-todo'

In [ ]:
response = cloudformation.describe_stacks(
    StackName=cloudfront_stack_name
)

print response

outputs = response['Stacks'][0]['Outputs']
distro_id =  [d for d in outputs if d['OutputKey'] == 'CFDistID'][0]['OutputValue']
print distro_id

In [ ]:
response = cloudfront.get_distribution_config(
    Id=distro_id
)

In [ ]:
etag = response['ETag']
config = response['DistributionConfig']

In [ ]:
print config['CacheBehaviors']
for key in config['CacheBehaviors']:
    print key

In [ ]:
lambdaAssociation = {'Quantity': 1, 'Items':[{'LambdaFunctionARN':cf_lambda_arn, 'EventType':'origin-request'}]}
print lambdaAssociation

In [ ]:
cacheBehaviorsItems = config['CacheBehaviors']['Items']
for cb in cacheBehaviorsItems:
    if cb['PathPattern'] == '/todos/*':
        cb['LambdaFunctionAssociations'] = lambdaAssociation

In [ ]:
print cacheBehaviorsItems

In [ ]:
config['CacheBehaviors']['Items'] = cacheBehaviorsItems

In [ ]:
print config

In [ ]:
update_response = cloudfront.update_distribution(
    DistributionConfig=config,
    Id=distro_id,
    IfMatch=etag
)

In [ ]:
update_response